In [1]:
import tensorflow as tf

In [2]:
class Generator:
    def __init__(self,depth=[1024,512,156,128],s_size=4):#每一层的filter的个数
        self.depth=depth+[3]  #最后一层是RGB层
        self.s_size=s_size
        self.reuse=False#将该参数设置为true是指只能从已经创建过的变量进行读取
    def __call__(self, inputs, training=False):
        inputs=tf.convert_to_tensor(inputs)#将变量转为tensor格式
        with tf.variable_scope('g',reuse=self.reuse):
            with tf.variable_scope('reshape'):
                outputs=tf.layers.dense(inputs,self.depth[0]*self.s_size*self.s_size)
                outputs=tf.reshape(outputs,[-1,self.s_size,self.s_size,self.depth[0]])#将数据进行reshape
                outputs=tf.nn.relu(tf.layers.batch_normalization(outputs,training=training),name="outputs")
            with tf.variable_scope('deconv1'):
                outputs=tf.layers.conv2d_transpose(outputs,self.depth[1],[5,5],strides=(2,2),padding='SAME')#[5,5]的kernel size
                outputs=tf.nn.relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('deconv2'):
                outputs=tf.layers.conv2d_transpose(outputs,self.depth[2],[5,5],strides=(2,2),padding='SAME')#[5,5]的kernel size
                outputs=tf.nn.relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('deconv3'):
                outputs=tf.layers.conv2d_transpose(outputs,self.depth[3],[5,5],strides=(2,2),padding='SAME')#[5,5]的kernel size
                outputs=tf.nn.relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('deconv4'):
                outputs=tf.layers.conv2d_transpose(outputs,self.depth[4],[5,5],strides=(2,2),padding='SAME')#[5,5]的kernel size
                outputs=tf.nn.relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('tanh'):
                outputs=tf.tanh(outputs,name='outputs')
        self.reuse=True
        self.variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='g')
        return outputs

In [4]:
class Discriminator:
    def __init__(self,depth=[64,128,256,512]):
        self.depth=[3]+depth
        self.reuse=False
    def __call__(self,inputs,training=False,name=''):
        def leaky_relu(x,leaky=0.2,name=''):
            return tf.maximum(x,x*leaky,name=name)#f(x)=αx，(x<0) f(x)=x，(x>=0)
        
        outputs=tf.convert_to_tensor(inputs)
        with tf.name_scope('d'+name),tf.variable_scope('d',reuse=self.reuse):
            with tf.variable_scope('conv1'):
                outputs=tf.layers.conv2d(outputs,self.depth[1],[5,5],strides=(2,2),padding='SAME')
                outputs=leaky_relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('conv2'):
                outputs=tf.layers.conv2d(outputs,self.depth[2],[5,5],strides=(2,2),padding='SAME')
                outputs=leaky_relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('conv3'):
                outputs=tf.layers.conv2d(outputs,self.depth[3],[5,5],strides=(2,2),padding='SAME')
                outputs=leaky_relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('conv4'):
                outputs=tf.layers.conv2d(outputs,self.depth[4],[5,5],strides=(2,2),padding='SAME')
                outputs=leaky_relu(tf.layers.batch_normalization(outputs,training=training),name='outputs')
            with tf.variable_scope('classify'):
                batch_size=outputs.get_shape()[0].value
                reshape=tf.reshape(outputs,[batch_size,-1])
                outputs=tf.layers.dense(reshape,2,name='outputs')
        self.reuse=True
        self.variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='d')
        return outputs

In [6]:
class DCGAN:
    def __init__(self,
                 batch_size=128,
                 s_size=4,z_dim=100,g_depth=[1024,512,256,128],d_depth=[64,128,256,512]):
        self.batch_size=batch_size
        self.s_size=s_size
        self.z_dim=z_dim
        self.g=Generator(depth=g_depth,s_size=self.s_size)
        self.d=Discriminator(depth=d_depth)
        self.z=tf.random_uniform([self.batch_size,self.z_dim],minval=-1.0,maxval=1.0)
    def loss(self,traindata):
        generated=self.g(self.z,training=True)
        g_outputs=self.d(generated,trainig=True,name='g')
        t_outputs=self.d(traindata,training=True,name='t')
        tf.add_to_collection('g_loss',tf._mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.ones([self.batch_size],dtype=tf.int64),
                                                                                              logits=g_outputs)))#计算loss，用softmax来算
        tf.add_to_collection('d_loss',tf.mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.ones([self.batch_size],dtype=tf.int64),
                                                                                             logits=t_outputs)))
        tf.add_to_collection('d_loss',tf.mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.zeros([self.batch_size],dtype=tf.int64),
                                                                                             logits=t_outputs)))
        return {
            self.g:tf.add_n(tf.get_collection('g_loss'),name='total_g_loss'),#添加所有输入张量。
            self.d:tf.add_n(tf.get_collection('d_loss'),name='total_d_loss')
        }
    def train(self,losses,learning_rate=0.0002,beta1=0.5):
        g_opt=tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1)
        d_opt=tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1)#优化迭代操作
        g_opt_op=g_opt.minimize(losses[self.g],var_list=self.g.variables)#最小化loss
        d_opt_op=d_opt.minimize(losses[self.d],var_list=self.d.variables)#self.variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='g')
        with tf.control_dependencies([g_opt_op,d_opt_op]):
            return tf.no_op('name=train')
        
    def sample_images(self,row=8,col=8,inputs=None):
        if inputs is None:
            inputs=self.z
        image=self.g(inputs,training=True)
        image=tf.image.convert_image_dtype(tf.div(tf.add(image,1.0),2.0),tf.uint8)
        image=[image for image in tf.split(image,self.batch_size,axis=0)]
        rows=[]
        for i in range(row):
            rows.append(tf.concat(image[col*i+0:col*i+col],2))
        image=tf.concat(rows,1)
        return tf.image.encode_jpeg(tf.squeeze(image,[0]))